# 1. Data Loading and Labeling

In [1]:
import pandas as pd
import numpy as np

In [2]:
true_df=pd.read_csv('True.csv')
fake_df=pd.read_csv('Fake.csv')
    

In [3]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


- Add a new column label:
    - 1 for real news (True.csv)
    - 0 for fake news (Fake.csv)

In [5]:
true_df['label']=1
fake_df['label']=0

### Combine both datasets into one and shuffle.

In [6]:
df = pd.concat([true_df, fake_df], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  

df.head()

,title,text,subject,date,label
0,BREAKING: D.C. Driver Plows Into Capitol Polic...,It happened around 9:30 a.m. at the 100 block ...,Government News,"Mar 29, 2017",0
1,(VIDEO) Female College Students Protesting Bec...,21st Century Wire says US college students con...,US_News,"November 11, 2016",0
2,"Republican senators criticize CBO, welcome hea...",WASHINGTON (Reuters) - Senate Majority Leader ...,politicsNews,"March 14, 2017",1
3,China's Xi tells Britain's May North Korea iss...,BEIJING (Reuters) - The North Korean issue sho...,worldnews,"September 25, 2017",1
4,Trump allies say Mueller unlawfully obtained t...,WASHINGTON (Reuters) - An organization establi...,politicsNews,"December 16, 2017",1


## 2. Understanding the Data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


- **Checkig for missing values**

In [8]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

#### `Observation` :  
- No missing values in the data

- **Checking for duplicates**

In [9]:
df.duplicated().sum()

209

#### `Observation` :  
- There are 209 duplicates presents in the data which are needed to remove.

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df.duplicated().sum()

0

#### `Observation` 
- Duplicates are removed.

- **Inspect a few rows of fake and true news**

In [13]:
df[df["label"] == 1].head()

,title,text,subject,date,label
2,"Republican senators criticize CBO, welcome hea...",WASHINGTON (Reuters) - Senate Majority Leader ...,politicsNews,"March 14, 2017",1
3,China's Xi tells Britain's May North Korea iss...,BEIJING (Reuters) - The North Korean issue sho...,worldnews,"September 25, 2017",1
4,Trump allies say Mueller unlawfully obtained t...,WASHINGTON (Reuters) - An organization establi...,politicsNews,"December 16, 2017",1
6,Stampede in India's financial capital kills at...,MUMBAI (Reuters) - A rush hour stampede killed...,worldnews,"September 29, 2017",1
7,"Despite carbon ruling, White House says U.S. c...",WASHINGTON (Reuters) - The White House promise...,politicsNews,"February 10, 2016",1


In [14]:
df[df['label']==0].head()

,title,text,subject,date,label
0,BREAKING: D.C. Driver Plows Into Capitol Polic...,It happened around 9:30 a.m. at the 100 block ...,Government News,"Mar 29, 2017",0
1,(VIDEO) Female College Students Protesting Bec...,21st Century Wire says US college students con...,US_News,"November 11, 2016",0
5,BREAKING: Ecuadorian Embassy Admits They Cut W...,The Ecuadorian government admitted on Tuesday ...,left-news,"Oct 19, 2016",0
10,Henningsen on CrossTalk Debating ‘Trump & His ...,Has President Trump handed over US foreign pol...,Middle-east,"September 15, 2017",0
11,BOMBSHELL: Trump Is Being Investigated By SIX...,You can probably expect Donald Trump to contin...,News,"January 18, 2017",0


In [15]:
df['label'].value_counts()

label
0    23478
1    21211
Name: count, dtype: int64

#### `Observation` : 
- Here we can get the review that 0 (fake) news are 23478 and 1 (true) news are 21211

## 3.Data Cleaning and Preprocessing

In [16]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import warnings


warnings.filterwarnings('ignore')  # Ignore warnings

# Download required NLTK data
for pkg in ['stopwords', 'punkt', 'wordnet', 'omw-1.4']:
    nltk.download(pkg, quiet=True)

# Initialize resources
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess(raw_text, flag='lemma'):

    # Step 1: Clean text (remove non-alphabets, lowercasing)
    sentence = re.sub("[^a-zA-Z]", " ", str(raw_text))
    sentence = sentence.lower()
    
    # Step 2: Tokenize
    tokens = sentence.split()
    
    # Step 3: Remove stopwords
    clean_tokens = [t for t in tokens if t not in stop_words]
    
    # Step 4: Stemming or Lemmatization
    if flag == 'stem':
        clean_tokens = [stemmer.stem(word) for word in clean_tokens]
    else:
        clean_tokens = [lemmatizer.lemmatize(word) for word in clean_tokens]
    
    # Step 5: Return joined text and token count
    return pd.Series([" ".join(clean_tokens), len(clean_tokens)])


df['content'] = df['title'].fillna('') + " " + df['text'].fillna('')  # Combine title + text

df[['clean_content', 'token_count']] = df['content'].apply(lambda x: preprocess(x, flag='lemma'))  # Apply preprocessing

df[['content', 'clean_content', 'token_count']].head()  # Preview

,content,clean_content,token_count
0,BREAKING: D.C. Driver Plows Into Capitol Polic...,breaking c driver plow capitol police shot fir...,48
1,(VIDEO) Female College Students Protesting Bec...,video female college student protesting trump ...,121
2,"Republican senators criticize CBO, welcome hea...",republican senator criticize cbo welcome healt...,71
3,China's Xi tells Britain's May North Korea iss...,china xi tell britain may north korea issue pe...,34
4,Trump allies say Mueller unlawfully obtained t...,trump ally say mueller unlawfully obtained tho...,392


#### `Observation` : 
- Here now our data is cleaned.
- This is our cleaned dataset

In [17]:
df.to_csv("Fake_news_Detection.csv")
